In [5]:
import numpy as np
import pandas as pd
import os
import torch
import time,sys
from transformers import AlbertForQuestionAnswering, AlbertTokenizer, AlbertTokenizerFast

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [17]:
tokenizer_address = '"ai4bharat/indic-bert"/indic-bert-v1'
model_address = '"ai4bharat/indic-bert"/indic-bert-v1'

In [9]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135M [00:00<?, ?B/s]

In [10]:
tokenizer.model_max_length = 512 # only for ALBERT

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
def preprocess_data(data):
    encodings = tokenizer(list(data["context"]), list(data["question"]), truncation=True, padding=True)

    start_positions = []
    end_positions = []
    id_ = []
    for i in range(len(data["answer_start"])):
        start_positions.append(encodings.char_to_token(i,data["answer_start"][i]))
        end_positions.append(encodings.char_to_token( i,(data["answer_start"][i] + len(data['answer_text'][i]) - 1) ))
        
            
        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
        
        
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    
    return encodings

In [14]:
train_encodings = preprocess_data(train)
dev_encodings = tokenizer(list(test["context"]), list(test["question"]),max_length=512, truncation=True, padding=True)

In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW

class chaiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)
    
train_dataset = chaiDataset(train_encodings)
dev_dataset = chaiDataset(dev_encodings)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=1, shuffle=False)

In [22]:
model = AlbertForQuestionAnswering.from_pretrained('ai4bharat/indic-bert')

model.to(device)



Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForQuestionAnswering: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly

AttributeError: 'AlbertForQuestionAnswering' object has no attribute 'bert'